In [10]:
import pandas as pd
import numpy as np
import myfitnesspal
import datetime


from flask import g, Flask, render_template, request, session, redirect, url_for, jsonify
from werkzeug.wrappers import Request, Response

from werkzeug.serving import run_simple

# MyFitnessPal

In [11]:
client = myfitnesspal.Client('groupb3', password='groupb3thebest')

In [12]:
date = datetime.date.today()
day = client.get_date(date)
print(day)
print(day.exercises[0].get_as_list())

calories = 0
#calories = day.exercises[0].get_as_list()[0]['nutrition_information']['calories burned']
print('You burned ' + str(calories) + ' calories today')

03/11/22 {}
[]
You burned 0 calories today


In [13]:
current_weight = client.get_measurements(lower_bound = datetime.date.today())
print('Your current weight is: ' + str(current_weight) + 'kg')

Your current weight is: OrderedDict()kg


# WebApp

In [42]:
lng = 4.953
lat = 52.356

In [43]:
content_string = 'BLABLABLA'

In [44]:
app = Flask(__name__)


In [45]:
@app.route("/")
def index(name=None):
    return render_template('map.html', lng = lng, lat = lat, content_string = content_string)

In [46]:
@app.route("/map")
def maps():
    return render_template('map.html', lng = lng, lat = lat, content_string = content_string)

In [47]:
@app.route("/profile", methods = ['POST', 'GET'])
def profile(calories = calories, current_weight = current_weight):
    user_name = request.form.get('exampleInputEmail1')
    print(user_name)
    return render_template('profile.html', calories = calories, current_weight = current_weight)

In [ ]:
run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Mar/2022 10:58:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2022 10:58:23] "GET /images/icon.png HTTP/1.1" 404 -
127.0.0.1 - - [11/Mar/2022 10:58:23] "GET /images/icon.png HTTP/1.1" 404 -
127.0.0.1 - - [11/Mar/2022 10:58:23] "GET /images/icon.png HTTP/1.1" 404 -
